In [2]:
%run "..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

import h5py
from ImageAnalysis3.classes import _allowed_kwds
import ast

21868


In [3]:
import numpy as np
from cellpose import models, utils
import os
import sys
import torch
import cv2
import h5py 
import ast

import pandas as pd
sys.path.append(r"..\..\..\..\Documents")
from ImageAnalysis3.io_tools.crop import generate_neighboring_crop
from skimage.segmentation import random_walker

channel_2_th = {
    '750':1000,
    '647':1500,
}
overwrite=False
torch.cuda.set_device('cuda:0')

print(os.getpid())

2021-12-07 23:49:55,396 [INFO] WRITING LOG OUTPUT TO C:\Users\puzheng\.cellpose\run.log
21868


In [4]:
save_folder = r'\\hubel\Hubel_SSD\Pu\20211129-P_brain_CTP12-RNA'
save_filenames = [os.path.join(save_folder, _fl) for _fl in os.listdir(save_folder)
                  if _fl.split(os.extsep)[-1]=='hdf5']
print(len(save_filenames))

125


In [5]:
gene_readout_df = pd.read_excel(r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-12_mop_markers\smFISH_cell_class\MOP_marker_gene_readout.xlsx')
#gene_readout_df

In [6]:
from ImageAnalysis3 import meta_tools
reload(ia.meta_tools)

<module 'ImageAnalysis3.meta_tools' from 'E:\\Users\\puzheng\\Documents\\ImageAnalysis3\\meta_tools\\__init__.py'>

In [7]:
# generate metadata

import multiprocessing as mp

save_metadata = True
overwrite = False
# prepare args
metadata_args = []

processed_fovs = []

for _filename in save_filenames:
    
    with h5py.File(_filename, "r", libver='latest') as _f:
        
        fov_id = _f.attrs['fov_id']
        processed_fovs.append(fov_id)
        parameters = ast.literal_eval(_f.attrs['shared_parameters']) 
        
    pixel_sizes = parameters['distance_zxy']
    single_im_size = parameters['single_im_size']

    segmentation_filename = _filename.replace('.hdf5', '_Segmentation.npy')
    
    metadata_filename = _filename.replace('.hdf5', '_CellLocations.csv')
    
    metadata_args.append((segmentation_filename, fov_id, #np.array(fov_positions.iloc[fov_id]), 
                          single_im_size, pixel_sizes, 
                          save_metadata, metadata_filename, overwrite) )
    

In [8]:
%%time

from ImageAnalysis3.meta_tools import cell_locations
reload(ia.meta_tools.cell_locations)

with mp.Pool(40) as _meta_pool:
    print("Start")
    # Multi-proessing!
    _meta_pool.starmap(
        cell_locations.SegmentationMask3D_2_CellLocations,
        metadata_args, 
        chunksize=1)
    # close multiprocessing
    _meta_pool.close()
    _meta_pool.join()
    _meta_pool.terminate()

Start
Wall time: 4.62 s


In [9]:
from ImageAnalysis3.meta_tools import cell_locations
reload(ia.meta_tools.cell_locations)

<module 'ImageAnalysis3.meta_tools.cell_locations' from 'E:\\Users\\puzheng\\Documents\\ImageAnalysis3\\meta_tools\\cell_locations.py'>

In [10]:
import json
#microscope_filename = r'E:\Users\puzheng\Documents\Merfish_analysis\Merfish_Analysis_Scripts\merlin_parameters\microscope\storm6_microscope.json'
#microscope_info = #json.load(open(microscope_filename, 'r'))

microscope_info = {'flip_horizontal': True, 'flip_vertical': False, 'transpose': True, 'microns_per_pixel': 0.108}
print(microscope_info)

{'flip_horizontal': True, 'flip_vertical': False, 'transpose': True, 'microns_per_pixel': 0.108}


In [11]:
cell_location_filenames = [os.path.join(save_folder, _fl) for _fl in os.listdir(save_folder)
                        if "_CellLocations.csv" in _fl]
print(len(cell_location_filenames))

125


In [12]:
from ImageAnalysis3.meta_tools.global_alignments import Load_PositionFile

fov_positions = Load_PositionFile(r'\\10.245.74.158\Chromatin_NAS_1\20211129-P_brain_CTP12-RNA\Experiments\positions_all.txt')
fov_positions

,x,y
0,-4250.40,-3841.20
1,-4250.40,-4051.20
2,-4250.40,-4261.20
3,-4250.40,-4471.20
4,-4250.40,-4681.20
...,...,...
388,-53.67,-2568.62
389,-295.20,-2806.31
390,-306.70,-3028.67
391,-321.70,-3290.20


In [13]:
from ImageAnalysis3.meta_tools import cell_locations
reload(ia.meta_tools.cell_locations)

merged_cell_filename = os.path.join(save_folder, f'merged_cell_locations_{min(processed_fovs)}-{max(processed_fovs)}.csv')
print(merged_cell_filename)
merged_cell_locations = cell_locations.Merge_CellLocations(cell_location_filenames, 
                                                           microscope_info,
                                                           fov_positions,
                                                           save=True, save_filename=merged_cell_filename,
                                                           overwrite=True,)

\\hubel\Hubel_SSD\Pu\20211129-P_brain_CTP12-RNA\merged_cell_locations_0-124.csv
- Start merging 125 cell locations
2021-12-07 23:50:55,166 [INFO] Note: NumExpr detected 44 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-12-07 23:50:55,167 [INFO] NumExpr defaulting to 8 threads.
-- 12604 cells converted into MetaData
-- save 12604 cells into file:\\hubel\Hubel_SSD\Pu\20211129-P_brain_CTP12-RNA\merged_cell_locations_0-124.csv
-- merge cell-locations in 1.716s
